In [189]:
import json

### LOAD ###
#args = [json.loads(ln) for ln in open("../data/cmv_processed.jsonl")]
arg_ids = [json.loads(ln)["id"] for ln in open("../data/cmv_processed.jsonl")]
args = [json.loads(ln)["argument"]["argument"] for ln in open("../data/cmv_processed.jsonl")]
claims = [json.loads(ln)["claim"] for ln in open("../data/cmv_processed.jsonl")]

extracts = [json.loads(ln)["extract"] for ln in open("../data/argument_extracts.jsonl")]
extract_ids = [json.loads(ln)["id"] for ln in open("../data/argument_extracts.jsonl")]

FileNotFoundError: [Errno 2] No such file or directory: '../data/cmv_processed.jsonl'

In [3]:
import os
path = "/Users/joshua.sheppard/PycharmProjects/countaBot/"
os.chdir(path)

import json
from summarizer import Summarizer
from src.utils_.utils import sentences_segment
from src.detection.BERT_adu_classifier import predict

claim


In [96]:
model = Summarizer()
summarised = model(arg, num_sentences = 4, min_length=0)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [161]:
_ = 523

arg = args[_]
claim = claims[_]

print(arg, "\n")
print(claim, "\n")

Help me understand why anyone would be supporting illegal activities. Many presidents on both sides have talked about the need for stronger boarder controls. The data shows that legal immigration rates are steadily increasing and the annual number of immigrants are higher than at any other point in US history. This as well as other things leads me to believe there is an acceptable and effective path to enter the United States legally. Illegal immigration results in a burden on our economy taxes health systems... 

There is no good reason to support illegal immigration and sanctuary cities. 



In [165]:
adu_list = []
for sent in sentences_segment(arg):
    if predict(sent) == "claim":
        print(sent)
        continue
    else:
        adu_list.append(sent)

This as well as other things leads me to believe there is an acceptable and effective path to enter the United States legally.


In [166]:
claim

'There is no good reason to support illegal immigration and sanctuary cities.'

In [167]:
# Coreference
adu_list

['Help me understand why anyone would be supporting illegal activities.',
 'Many presidents on both sides have talked about the need for stronger boarder controls.',
 'The data shows that legal immigration rates are steadily increasing and the annual number of immigrants are higher than at any other point in US history.',
 'Illegal immigration results in a burden on our economy taxes health systems...']

In [184]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer

kb = KeyBERT()

def extract_keyphrase(doc, n_gram=3, n_kp=3, use_mmr="False", use_maxsum="False"):
    #kp = kb.extract_keywords(doc, vectorizer=KeyphraseCountVectorizer(), stop_words="english", diversity=0.2,)
    kp = kb.extract_keywords(doc, keyphrase_ngram_range=(1, 3), diversity=0.2, stop_words=None, use_mmr=True)
    #kp_ = kb.extract_keywords(doc, keyphrase_ngram_range=(1, 2), diversity=0.2, stop_words=None, use_mmr=True)

    return [i[0] for i in kp]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [185]:
adu_kps = []
claims_kps = extract_keyphrase(claim)

for adu in adu_list:
    adu_kps.append(extract_keyphrase(adu))

In [186]:
adu_kps
#claims_kps

[['supporting illegal activities',
  'be supporting illegal',
  'supporting illegal',
  'illegal activities',
  'why anyone would'],
 ['stronger boarder controls',
  'many presidents on',
  'boarder controls',
  'for stronger boarder',
  'sides have talked'],
 ['immigration rates are',
  'legal immigration rates',
  'immigration rates',
  'immigrants are higher',
  'number of immigrants'],
 ['economy taxes health',
  'taxes health systems',
  'taxes health',
  'our economy taxes',
  'illegal immigration results']]

In [170]:
from src.utils_.elastic_db import ElasticDB

# TODOs: Implement KW Selection
# INIT DB OBJECT
PORT = "http://localhost:9200"
db = ElasticDB(elastic_port=PORT)

INFO:src.utils_.elastic_db:Connecting to http://localhost:9200 
INFO:src.utils_.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 


In [187]:
query = " ".join(i for i in claims_kps)
print(adu_kps[1])
search = [(i["_source"]["document"]["source"], i["_source"]["document"]["text"]) for i in db.search(query_=query, k=5)]

INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.069s]


['stronger boarder controls', 'many presidents on', 'boarder controls', 'for stronger boarder', 'sides have talked']


In [188]:
search

[('wikipedia',
  "That same year, Gonzalez opposed a proposed state law banning sanctuary cities for illegal immigrants, and requiring local officials to cooperate with federal immigration authorities. In 2018, Gonzalez criticized US immigration policy, saying 'children should not be in immigration detention', and 'I do not support #ICERaids that threaten to deport millions of undocumented immigrants'. In 2019, Gonzalez barred his officers from participating in raids by ICE to detain and deport illegal immigrants."),
 ('common_crawl_news',
  'End illegal immigration. End sanctuary cities. Make our country safer.'),
 ('wikipedia',
  "The event promoted an inclusive theology, meant to reassure the LGBT community that they have Christian support. Illegal Immigration Controversy. In 2007 St Luke's made national headlines by providing sanctuary, in the face of strong governmental and outside pressure, to an illegal immigrant hoping to keep her children in America."),
 ('wikipedia',
  "The t

In [32]:
from tqdm.notebook import tqdm
import multiprocessing
from src.detection.stance_classifier import sentence_stance, compare_stance
from src.utils_.word_net_expansion import expand_query
from src.detection.stance_classifier import sentence_stance
# from multiprocessing.pool import ThreadPool as Pool
import time

# Disable Huggingface Logging
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### RETRIEVER ###
db = db
queries = []
retrieved_ev = []

topic_ids = [json.loads(ln)["id"] for ln in open("../data/argument_topic_concept.jsonl")]
concept_ids = [json.loads(ln)["id"] for ln in open("../data/argument_concept.jsonl")]

# TODOs: Argumentative Sentence
# TODOs: Query Expansion
def search(mined):
    id_ = mined["id"]
    # print("\n", id_)

    topic = arg["argument"][0]["topic"]
    concept = arg["argument"][0]["concept"]

    retrieve_len = 5
    retrieved = []

    # for adu in mined["argument"]:
    for adu in mined["tgt_counter"]:

        sentence = adu["sentence"]
        if len(tokeniser(sentence)) <= 8:
            continue

        kp = list(set(adu["kp"]))
        #print(kp)
        # topic = adu["topic"]
        # concept = adu["concept"]

        kp.append(topic) if topic else kp
        kp.append(concept) if concept else kp
        # print(kp)

        query = ", ".join(i for i in kp)

        search = [(i["_source"]["document"]["source"], i["_source"]["document"]["text"]) for i in db.search(query_=query, k=retrieve_len)]

        source = [i[0] for i in search]
        evidence = [i[1] for i in search]

        merged = ", ".join(i for i in evidence)
        ev_kp = list(set(yake_extract_keyphrase(merged)))

        retrieved.append({"passages": evidence, "kp": [i for i in ev_kp], "source": source})

    # TODOs: Implement yield without storing list
    return ({
        "id": id_,
        "argument": mined["argument"],
        "tgt_counter": mined["tgt_counter"],
        "retrieved": [i for i in retrieved],
    })

# SINGLE ARGUMENT INSPECT
# SAMPLE = unique_args[0]
# results = search(SAMPLE)

tic = time.time()
SAMPLE = unique_args[0:100]

retrieved_ev = []
with tqdm(total=(len(SAMPLE)), position=0, leave=True) as pbar:
    for arg in SAMPLE:
        retrieved_ev.append(search(arg))
        pbar.update()
toc = time.time()

  0%|          | 0/100 [00:00<?, ?it/s]

INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.072s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.084s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.070s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.069s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.045s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.105s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.038s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.051s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.025s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.037s]
INFO:elast